In [1]:
import os
import re
import sys
sys.path.append('..')

import numpy as np
import pandas as pd

from preprocessing import get_dataframe

In [2]:
df = get_dataframe(os.path.join('..', 'data', 'lyrics'))

In [3]:
df.head()

,album,song,text
0,AHardDaysNight,Ill_Be_Back.txt,"You know, if you break my heart I'll go But I'..."
1,AHardDaysNight,Cant_Buy_Me_Love.txt,"Can't buy me love, oh Love, oh Can't buy me lo..."
2,AHardDaysNight,Any_Time_At_All.txt,Any time at all Any time at all Any time at al...
3,AHardDaysNight,A_Hard_Days_Night.txt,It's been a hard day's night And I've been wor...
4,AHardDaysNight,Ill_Cry_Instead.txt,I've got every reason on earth to be mad 'Caus...


In [4]:
df.describe()

,album,song,text
count,182,182,182
unique,13,180,180
top,TheBeatles,All_You_Need_Is_Love.txt,"Love, love, love Love, love, love Love, love, ..."
freq,30,2,2


In [5]:
df.text.nunique()

180

In [6]:
df.song.nunique()

180

#### So, we have 2 duplicate songs in 2 separate albums

In [7]:
df_gb_song = df.groupby('song').count()

In [8]:
df_gb_song[df_gb_song.text != 1]

,album,text
song,,
All_You_Need_Is_Love.txt,2,2
Yellow_Submarine.txt,2,2


In [9]:
df[df.song == 'All_You_Need_Is_Love.txt']

,album,song,text
73,MagicalMysteryTour,All_You_Need_Is_Love.txt,"Love, love, love Love, love, love Love, love, ..."
165,YellowSubmarine,All_You_Need_Is_Love.txt,"Love, love, love Love, love, love Love, love, ..."


In [10]:
df[df.song == 'Yellow_Submarine.txt']

,album,song,text
67,Revolver,Yellow_Submarine.txt,In the town where I was born Lived a man who s...
167,YellowSubmarine,Yellow_Submarine.txt,In the town where I was born Lived a man who s...


### We have to consider this fact when building models -> to not use same texts twice!

#### Just for fun: let's count frequencies of search pattern words in one of the songs

In [11]:
song = list(df[(df['song'] == 'All_You_Need_Is_Love.txt') & (df['album'] == 'MagicalMysteryTour')].text)[0]

In [12]:
rep = {",": "", ".": "", "(": "", ")": ""} # define desired replacements here
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
song = pattern.sub(lambda m: rep[re.escape(m.group(0))], song)

In [13]:
words = np.array([word.lower() for word in song.split(' ')])

In [14]:
unique, counts = np.unique(words, return_counts=True)
freq_df = pd.DataFrame({"word": unique, "count": counts})
freq_df.sort_values('count', ascending=False).head(10)

,word,count
22,love,76
56,you,70
1,all,57
14,is,56
27,need,56
4,can,11
0,,9
20,lo-o-ove,9
42,that,7
29,nothing,7


#### Considering the fact, we need to search for pattern with words 'all' 'is' 'love', this song will have a huge benefit even in the frequency terms (without context or so). Much more interesting will be to look on places 2 and 3.